In [9]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#!pip uninstall tensorflow

In [ ]:
#!pip install tensorflow==2.5.0

In [ ]:
!pip list

Package                       Version            
----------------------------- -------------------
absl-py                       0.12.0             
alabaster                     0.7.12             
albumentations                0.1.12             
altair                        4.1.0              
appdirs                       1.4.4              
argon2-cffi                   20.1.0             
arviz                         0.11.2             
astor                         0.8.1              
astropy                       4.2.1              
astunparse                    1.6.3              
async-generator               1.10               
atari-py                      0.2.9              
atomicwrites                  1.4.0              
attrs                         21.2.0             
audioread                     2.1.9              
autograd                      1.3                
Babel                         2.9.1              
backcall                      0.2.0              


In [10]:
import os, glob
import matplotlib.pyplot as plt
import numpy as np
import pickle
import nibabel as nb
import matplotlib.image as mpimg
import cv2
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.utils import normalize
from tensorflow.keras.utils import to_categorical
from tqdm.notebook import tqdm
from sys import getsizeof
from tensorflow.keras.models import load_model

import nibabel as nib

In [11]:
model

NameError: ignored

In [12]:
tf.keras.backend.clear_session()
del model

NameError: ignored

In [ ]:
512/4

128.0

In [13]:
# Define constants

# Path variables
data_dir = '/content/drive/Shareddrives/Banana Leaf Development/Data/Couinaud_Annotation_Data/'
data_dir_train_image = os.path.join(data_dir, 'data_splits_preprocessed/training/img:-1000:1000/img/')
data_dir_train_mask = os.path.join(data_dir,  'data_splits_preprocessed/training/couinaud/img/')

data_dir_test_image = os.path.join(data_dir, 'data_splits_preprocessed/testing/img:-1000:1000/img/')
data_dir_test_mask = os.path.join(data_dir, 'data_splits_preprocessed/testing/couinaud/img/')

data_dir_validate_image = os.path.join(data_dir, 'data_splits_preprocessed/validate/img:-1000:1000/img/')
data_dir_validate_mask = os.path.join(data_dir, 'data_splits_preprocessed/validate/couinaud/img/')

class_names = ['class_0','class_1','class_2','class_3',
               'class_4','class_5','class_6','class_7','class_8']

# Training variables
SEED = 7
BATCH_SIZE_TRAIN = 10
BATCH_SIZE_TEST = 10
BATCH_SIZE_VALIDATE = 10
NUM_TRAIN = 9288
NUM_TEST = 1042
NUM_VALIDATE = 1207

N_CLASSES = 9


# Image variables
IMAGE_HEIGHT = int(512/4)
IMAGE_WIDTH = int(512/4)
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)


#Original Image size
OG_IMAGE_HEIGHT = 512
OG_IMAGE_WIDTH = 512
OG_IMG_SIZE = (OG_IMAGE_HEIGHT, OG_IMAGE_WIDTH)

NUM_EPOCH = 100

In [14]:
### Import the 'data' and save them in lists 

#Capture training image info as a list
scaled_train_images = []

for index, img_path in tqdm(enumerate(sorted(glob.iglob(data_dir_train_image+ '*.png')))):
  substring = img_path[-5:]

  img = cv2.imread(img_path, 0)       
  scaled_train_images.append(cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_LINEAR))

#Convert list to array for machine learning processing        
#train_images = np.array(train_images)
scaled_train_images = np.array(scaled_train_images)

print('##### TRAINING IMAGES #####')
print(scaled_train_images.shape)
#print(type(train_images[0,0,0]))
##############################################################

#Capture mask/label info as a list
scaled_train_masks = []

for index, img_path in tqdm(enumerate(sorted(glob.iglob(data_dir_train_mask + '*.tif')))):
  substring = img_path[-5:]

  img = cv2.imread(img_path, 0)       
  scaled_train_masks.append(cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_LINEAR))

#Convert list to array for machine learning processing      
scaled_train_masks = np.array(scaled_train_masks)

print('##### TRAINING MASKS #####') 
print(scaled_train_masks.shape)
#print(type(train_masks[0,0,0]))
##############################################################
##############################################################

#Capture validate image info as a list
scaled_validate_images = []

for index, img_path in tqdm(enumerate(sorted(glob.iglob(data_dir_validate_image+ '*.png')))):
  substring = img_path[-5:]

  img = cv2.imread(img_path, 0)       
  scaled_validate_images.append(cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_LINEAR))

#Convert list to array for machine learning processing        
scaled_validate_images = np.array(scaled_validate_images)

print('##### VALIDATE IMAGES #####')
print(scaled_validate_images.shape)
#print(type(validate_images[0,0,0]))
##############################################################

#Capture mask/label info as a list
scaled_validate_masks = []

for index, img_path in tqdm(enumerate(sorted(glob.iglob(data_dir_validate_mask + '*.tif')))):
  substring = img_path[-5:]

  img = cv2.imread(img_path, 0)       
  scaled_validate_masks.append(cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_LINEAR))





#Convert list to array for machine learning processing      
scaled_validate_masks = np.array(scaled_validate_masks)

print('##### VALIDATE MASKS #####') 
print(scaled_validate_masks.shape)
#print(type(validate_masks[0,0,0]))
##############################################################


##### TRAINING IMAGES #####
(9288, 128, 128)



##### TRAINING MASKS #####
(9288, 128, 128)



##### VALIDATE IMAGES #####
(1207, 128, 128)



##### VALIDATE MASKS #####
(1207, 128, 128)


In [15]:
from sklearn.utils import shuffle
scaled_train_images, scaled_train_masks = shuffle(scaled_train_images,scaled_train_masks, random_state = SEED)


In [16]:
scaled_train_images.shape

(9288, 128, 128)

In [17]:

round(getsizeof(scaled_train_images) / 1024 / 1024 / 1024 ,2) #to get GB 

0.14

In [ ]:
# np.unique(train_masks)

In [18]:
# train_images = np.expand_dims(train_images, axis=3)
# test_images = np.expand_dims(test_images, axis=3)
# validate_images = np.expand_dims(validate_images, axis=3)

scaled_train_images = np.expand_dims(scaled_train_images, axis=3)
scaled_validate_images = np.expand_dims(scaled_validate_images, axis=3)


# train_images = normalize(train_images, axis=1) #Thia will change the value from uint8 -> float64
# test_images = normalize(test_images, axis=1) #Thia will change the value from uint8 -> float64
# validate_images = normalize(validate_images, axis=1) #Thia will change the value from uint8 -> float64

#scaled_train_images = normalize(scaled_train_images, axis=1) #Thia will change the value from uint8 -> float64
#scaled_test_images = normalize(scaled_test_images, axis=1) #Thia will change the value from uint8 -> float64
#scaled_validate_images = normalize(scaled_validate_images, axis=1) #Thia will change the value from uint8 -> float64

print('#### IMAGES ###')
# print(train_images.shape)
print(scaled_train_images.shape)
print(type(scaled_train_images[0,0,0,0]))


#print(validate_images.shape)
print(scaled_validate_images.shape)
print(type(scaled_validate_images[0,0,0,0]))

##########################################################################
# train_masks = np.expand_dims(train_masks, axis=3)
# test_masks = np.expand_dims(test_masks, axis=3)
# validate_masks = np.expand_dims(validate_masks, axis=3)

scaled_train_masks = np.expand_dims(scaled_train_masks, axis=3)
scaled_validate_masks = np.expand_dims(scaled_validate_masks, axis=3)

print('#### MASKS ###')
# print(train_masks.shape)
print(scaled_train_masks.shape)

# print(validate_masks.shape)
print(scaled_validate_masks.shape)

#### IMAGES ###
(9288, 128, 128, 1)
<class 'numpy.uint8'>
(1207, 128, 128, 1)
<class 'numpy.uint8'>
#### MASKS ###
(9288, 128, 128, 1)
(1207, 128, 128, 1)


In [ ]:
#Check how many classes are in the training masks


# all_classes = []
# for index, img_path in enumerate(sorted(glob.iglob(data_dir_train_mask + '*.tif'))):
#     img = cv2.imread(img_path, 0)   
#     classes_in_img = np.unique(img)
#     all_classes.append(classes_in_img)

# all_classes = [item for sublist in all_classes for item in sublist]

# class_counts = []
# print('Number of Instances')
# for i in range(0,9):
#   count = all_classes.count(i)
#   print('class ', i, ': ', count)
#   class_counts.append(count)

# plt.figure(figsize = (7,7))
# plt.bar(class_names, class_counts)
# plt.show()


In [ ]:
# from sklearn.utils import class_weight
# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(train_masks.reshape(-1)),
#                                                  train_masks.reshape(-1))
# print("Class weights are...:", class_weights)

In [ ]:
# class_weights_dict = {}
# for i in range(0,len(class_weights)):
#   class_weights_dict[i] = class_weights[i]

In [ ]:
# class_weights_dict

In [19]:
#Encode labels for all masks
#train_masks = to_categorical(train_masks, num_classes=N_CLASSES)
scaled_train_masks = to_categorical(scaled_train_masks, num_classes=N_CLASSES)
#print(train_masks.shape)
print(scaled_train_masks.shape)

(9288, 128, 128, 9)


In [20]:
round(getsizeof(scaled_train_images) / 1024 / 1024 / 1024 ,2) #to get GB 

0.0

In [ ]:
# #test_masks = to_categorical(test_masks, num_classes=N_CLASSES)
# scaled_test_masks = to_categorical(scaled_test_masks, num_classes=N_CLASSES)
# #print(test_masks.shape)
# print(scaled_test_masks.shape)

In [21]:
#validate_masks = to_categorical(validate_masks, num_classes=N_CLASSES)
scaled_validate_masks = to_categorical(scaled_validate_masks, num_classes=N_CLASSES)
#print(validate_masks.shape)
print(scaled_validate_masks.shape)

(1207, 128, 128, 9)


In [22]:
def display(display_list):
  plt.figure(figsize = (15,15))
  title = ['Input Image', 'True Mask', 'Predicted Mask', 'Predicted Mask \nw/ Threshold']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]), cmap = 'Greys_r')
  plt.show()

In [ ]:
# def show_dataset(datagen, num = 1):
#   for i in range(0, num):
#     image,mask = next(datagen)
#     display([image[0],mask[0]])


In [ ]:
 #show_dataset(train_generator, 20)

In [23]:
def unet(n_levels, initial_features=32, n_blocks=2, kernel_size=3, pooling_size=2, in_channels=1, out_channels=1):
    inputs = keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, in_channels))
    x = inputs
    
    convpars = dict(kernel_size=kernel_size, activation='relu', padding='same')
    
    #downstream
    skips = {}
    for level in range(n_levels):
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
        if level < n_levels - 1:
            skips[level] = x
            x = keras.layers.MaxPool2D(pooling_size)(x)
            
    # upstream
    for level in reversed(range(n_levels-1)):
        x = keras.layers.Conv2DTranspose(initial_features * 2 ** level, strides=pooling_size, **convpars)(x)
        x = keras.layers.Concatenate()([x, skips[level]])
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
            
    # output
    activation = 'sigmoid' if out_channels == 1 else 'softmax'
    x = keras.layers.Conv2D(out_channels, kernel_size=1, activation=activation, padding='same')(x)
    
    return keras.Model(inputs=[inputs], outputs=[x], name=f'UNET-L{n_levels}-F{initial_features}')

In [ ]:
# def dice_coef(y_true, y_pred):
#     """
#     write our own dice_coeficcient metric
#     """
#     numerator = tf.reduce_sum(y_true * y_pred)
#     dice = 2 * numerator / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred))
#     return dice

In [24]:

checkpoint_filepath = '/content/drive/Shareddrives/Banana Leaf Development/Data/Couinaud_Annotation_Data/models/All_Slices/0524_5:22pm/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

In [25]:
# Batches per epoch
EPOCH_STEP_TRAIN = NUM_TRAIN // BATCH_SIZE_TRAIN
EPOCH_STEP_TEST = NUM_TEST // BATCH_SIZE_TEST
EPOCH_STEP_VALIDATE = NUM_VALIDATE // BATCH_SIZE_VALIDATE

model = unet(n_levels= 4, out_channels=9)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = [tf.keras.metrics.MeanIoU(num_classes=N_CLASSES)])
#model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [dice_coef])
#model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [soft_dice_loss])

In [26]:
model.summary()

Model: "UNET-L4-F32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 32) 9248        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 64, 64, 32)   0           conv2d_1[0][0]                   
________________________________________________________________________________________

In [27]:
history = model.fit(scaled_train_images, scaled_train_masks, 
                    batch_size = BATCH_SIZE_TRAIN, 
                    steps_per_epoch=EPOCH_STEP_TRAIN,  
                    validation_data=(scaled_validate_images, scaled_validate_masks), 
                    validation_steps=EPOCH_STEP_VALIDATE,
                    epochs = NUM_EPOCH,
                    callbacks=[model_checkpoint_callback])

Epoch 1/100
928/928 [==============================] - 51s 36ms/step - loss: 0.1586 - mean_io_u: 0.7672 - val_loss: 0.1072 - val_mean_io_u: 0.8043
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/Banana Leaf Development/Data/Couinaud_Annotation_Data/models/All_Slices/0524_5:22pm/assets
Epoch 2/100
928/928 [==============================] - 34s 36ms/step - loss: 0.0719 - mean_io_u: 0.8182 - val_loss: 0.0706 - val_mean_io_u: 0.7914
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/Banana Leaf Development/Data/Couinaud_Annotation_Data/models/All_Slices/0524_5:22pm/assets
Epoch 3/100
928/928 [==============================] - 33s 36ms/step - loss: 0.0508 - mean_io_u: 0.8347 - val_loss: 0.0511 - val_mean_io_u: 0.8771
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/Banana Leaf Development/Data/Couinaud_Annotation_Data/models/All_Slices/0524_5:22pm/assets
Epoch 4/100
928/928 [==============================] - 33s 36ms/step - loss: 0.0409 - mean_io_u: 

KeyboardInterrupt: ignored

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.vlines(11, 0, 0.14, colors = 'b', linestyles = 'dashed', label='Saved Best Model') # We can see from the training history above that
                                                      # best model was saved at epoch 5
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(linestyle='-')
plt.show()

acc = history.history['mean_io_u']
val_acc = history.history['val_mean_io_u']

plt.plot(epochs, acc, 'g', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.vlines(11, 0.76, 1, colors = 'b', linestyles = 'dashed', label='Saved Best Model') # We can see from the training history above that
                                                      # best model was saved at epoch 5
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(linestyle='-')
plt.show()    

In [ ]:
imported_model = load_model(checkpoint_filepath)

In [ ]:
#IOU
mask_pred=model.predict(scaled_test_images)
mask_pred_argmax=np.argmax(mask_pred, axis=3) #Convert the probability into a max value within the area \

#ex: lets say we have three labels/categories
# [0,0.1,0] [0,0.05,0] [0,0.9,0] -> [0,0,0] [0,0,0] [0,1,0]

In [ ]:
#CNeed the test masks but no encoding for calculating IOU
test_masks_no_encode = [] 
scaled_test_masks_no_encode = []
for index, img_path in enumerate(sorted(glob.iglob(data_dir_test_mask + '*.tif'))):
  substring = img_path[-5:]
  # if '1' in substring:
  #   continue
  # if '2' in substring:
  #   continue
  # if '3' in substring:
  #   continue
  # if '4' in substring:
  #   continue
  #print(img_path)
  img = cv2.imread(img_path, 0)       
  test_masks_no_encode.append(img)
  scaled_test_masks_no_encode.append(cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_LINEAR))

#Convert list to array for machine learning processing      
test_masks_no_encode = np.array(test_masks_no_encode)
scaled_test_masks_no_encode = np.array(scaled_test_masks_no_encode)

test_masks_no_encode = np.expand_dims(test_masks_no_encode, axis=3)
scaled_test_masks_no_encode = np.expand_dims(scaled_test_masks_no_encode, axis=3)

print(test_masks_no_encode.shape)
print(scaled_test_masks_no_encode.shape)

In [ ]:
from tensorflow.keras.metrics import MeanIoU
IOU = MeanIoU(num_classes=N_CLASSES)  
IOU.update_state(scaled_test_masks_no_encode[:,:,:,0], mask_pred_argmax)
print("Mean IoU =", IOU.result().numpy())

In [ ]:
def calculateClassIOU(cm, num_classes=9):
  #: IOU = true_positive / (true_positive + false_positive + false_negative)
  #cm: confusion matrix where rows are actual and cols are predicted 
  
  for i in range(0,num_classes):
    trueP = cm[i,i]
    falseP = 0
    falseN = 0

    for col in range(0,num_classes):
      if (i != col):
        falseP = falseP + cm[i,col]

    for row in range(0,num_classes):
      if (i != row):
        falseN = falseN + cm[row,i]

    IoU = trueP / (trueP + falseP + falseN)
    print("IoU for class,",i, " is: ", IoU)

In [ ]:
#Get confusion Matrix
values = np.array(IOU.get_weights()).reshape(N_CLASSES, N_CLASSES)
print(values)

In [ ]:
#To calculate I0U for each class...
calculateClassIOU(values, 9)

In [ ]:
for i in range(0,50):
  print("Instance in test dataset: ", i)
  display([scaled_test_images[i], scaled_test_masks_no_encode[i], np.expand_dims(mask_pred_argmax[i], axis=2)])
  print("==================================================================================================================")

In [ ]:
for i in range(0,50):
  print("Instance in test dataset: ", i)
  display([scaled_test_images[i], scaled_test_masks_no_encode[i], np.expand_dims(mask_pred_argmax[i], axis=2)])
  mask_pred_argmaxOGSHAPE = cv2.resize(mask_pred_argmax[i].astype('float32')  , OG_IMG_SIZE, interpolation = cv2.INTER_LINEAR)
  display([np.expand_dims(test_images[i], axis=2), test_masks_no_encode[i], np.expand_dims(mask_pred_argmaxOGSHAPE, axis=2)])
  print("==================================================================================================================")

In [ ]:
tf.keras.backend.clear_session()
del model
del model2 
del imported_model

In [ ]:
tf.keras.backend.clear_session()
del imported_model